### Zero Shot NLI system results & Evaluation

#### Approach
- Author: Sushma Anand Akoju, Email: sushmaakoju@arizona.edu
- The following approach is as per guidance from Mihai.
- Approach for NLI systems' predictions, as suggested by Mihai:
  1. Run NLI with full annotated data.
  2. If top prediction is other than Neutral, choose top and STOP.
  3. If Neutral, run NLI in the other direction. If top prediction if 
   Entailment, report RE; otherwise Neutral.


In [ ]:
! pip install sentence-transformers allennlp-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.5/464.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.4/776.4 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 49.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
!pip install pycm==3.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.6/597.6 kB 26.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/natural-logic/final-datasets/'
!pwd

/content/drive/MyDrive/Colab Notebooks/natural-logic/final-datasets
/content/drive/MyDrive/Colab Notebooks/natural-logic/final-datasets


In [ ]:
import pandas as pd
import os

path = "/content/drive/MyDrive/Colab Notebooks/natural-logic/final-datasets/block-wise-data/blocks-dec26/annotated/combined/"
filepath = os.path.join(path,'annotations_feb10_all.xlsx')
df = pd.read_excel(filepath)

df.head()

,Unnamed: 0,SICK_id,Premise,Hypothesis,Modifier,Premise/Hypothesis/Both,Part of Premise/Hypothesis Modified,Ground Truth,GT,Quantifier Type
0,1,90,a man is jumping into an empty pool,a man is jumping into a full pool,NONE,NONE,NONE,Alternation,Alternation,NONE
1,2,90,every man is jumping into an empty pool,a man is jumping into a full pool,every,Premise,Subject,Alternation,Alternation,Universal
2,3,90,a man is jumping into an empty pool,every man is jumping into a full pool,every,Hypothesis,Subject,Negation,Negation,Universal
3,4,90,every man is jumping into an empty pool,every man is jumping into a full pool,every,Both,Subject,Negation,Negation,Universal
4,5,90,some man is jumping into an empty pool,a man is jumping into a full pool,some,Premise,Subject,Alternation,Alternation,Existential


In [ ]:
len(df), df['Ground Truth'].unique()

(1345,
 array(['Alternation', 'Negation', 'Neutral', 'Negation|Alternation', 'FE',
        'RE', 'Cover|FE', 'Equivalence', 'Cover'], dtype=object))

In [ ]:
label_map = {'FE':1, 'RE':2, 'Negation':3, 'Neutral':4,'Alternation':5, 'Cover':6,'Equivalence':7}

In [ ]:
# df['Ground Truth'][df['Ground Truth'] == 'Negation|Alternation'] = 

In [ ]:
# round1['label'] = round1['label'].map(label_map)
# round2['label'] = round2['label'].map(label_map)

In [ ]:
df.columns

Index(['Unnamed: 0', 'SICK_id', 'Premise', 'Hypothesis', 'Modifier',
       'Premise/Hypothesis/Both', 'Part of Premise/Hypothesis Modified',
       'Ground Truth', 'GT', 'Quantifier Type'],
      dtype='object')

In [ ]:
# set_129 = pd.concat([round1, round2, round3])
# set_129.head()

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
mod_tuple_list = [ (d['Premise'], d['Hypothesis']) for d in df.to_dict(orient='records')]
reversed_mod_tuple_list = [ ( d['Hypothesis'], d['Premise']) for d in df.to_dict(orient='records')]
reversed_mod_tuple_list[:1], mod_tuple_list[:1]

([('a man is jumping into a full pool',
   'a man is jumping into an empty pool')],
 [('a man is jumping into an empty pool',
   'a man is jumping into a full pool')])

In [ ]:
len(mod_tuple_list)

1345

## cross-encoder/nli-deberta-v3-base

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder

model = CrossEncoder('cross-encoder/nli-deberta-v3-base')
labels = []
label_mapping = ['contradiction', 'entailment', 'neutral']

for d in df.to_dict(orient='records'):
  forward_score = model.predict([(d['Premise'], d['Hypothesis'])]).argmax()
  rev_score = model.predict([(d['Hypothesis'], d['Premise'])]).argmax()
  if forward_score == 1 :
    labels.append("FE")
  elif forward_score == 0:
    labels.append("Negation")
  else:
    if rev_score == 1:
      labels.append("RE")
    else:
      labels.append("Neutral")
len(labels)

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/704M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/417 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


1345

In [ ]:
df['nli-deberta-v3-base'] = labels

In [ ]:
df.head()

,Unnamed: 0,SICK_id,Premise,Hypothesis,Modifier,Premise/Hypothesis/Both,Part of Premise/Hypothesis Modified,Ground Truth,GT,Quantifier Type,nli-deberta-v3-base
0,1,90,a man is jumping into an empty pool,a man is jumping into a full pool,NONE,NONE,NONE,Alternation,Alternation,NONE,Negation
1,2,90,every man is jumping into an empty pool,a man is jumping into a full pool,every,Premise,Subject,Alternation,Alternation,Universal,Negation
2,3,90,a man is jumping into an empty pool,every man is jumping into a full pool,every,Hypothesis,Subject,Negation,Negation,Universal,Negation
3,4,90,every man is jumping into an empty pool,every man is jumping into a full pool,every,Both,Subject,Negation,Negation,Universal,Negation
4,5,90,some man is jumping into an empty pool,a man is jumping into a full pool,some,Premise,Subject,Alternation,Alternation,Existential,Negation


## ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

max_length = 256

premise = "Two women are embracing while holding to go packages."
hypothesis = "The men are fighting outside a deli."

hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/albert-xxlarge-v2-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/electra-large-discriminator-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/xlnet-large-cased-snli_mnli_fever_anli_R1_R2_R3-nli"

tokenizer = AutoTokenizer.from_pretrained(hg_model_hub_name)
model = AutoModelForSequenceClassification.from_pretrained(hg_model_hub_name)

tokenized_input_seq_pair = tokenizer.encode_plus(premise, hypothesis,
                                                  max_length=max_length,
                                                  return_token_type_ids=True, truncation=True)

input_ids = torch.Tensor(tokenized_input_seq_pair['input_ids']).long().unsqueeze(0)
# remember bart doesn't have 'token_type_ids', remove the line below if you are using bart.
token_type_ids = torch.Tensor(tokenized_input_seq_pair['token_type_ids']).long().unsqueeze(0)
attention_mask = torch.Tensor(tokenized_input_seq_pair['attention_mask']).long().unsqueeze(0)

outputs = model(input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=None)
# Note:
# "id2label": {
#     "0": "entailment",
#     "1": "neutral",
#     "2": "contradiction"
# },

predicted_probability = torch.softmax(outputs[0], dim=1)[0].tolist()  # batch_size only one

print("Premise:", premise)
print("Hypothesis:", hypothesis)
print("Entailment:", predicted_probability[0])
print("Neutral:", predicted_probability[1])
print("Contradiction:", predicted_probability[2])


Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/703 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Premise: Two women are embracing while holding to go packages.
Hypothesis: The men are fighting outside a deli.
Entailment: 6.90110246068798e-05
Neutral: 0.0003960774920415133
Contradiction: 0.9995349645614624


In [ ]:
hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/albert-xxlarge-v2-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/electra-large-discriminator-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/xlnet-large-cased-snli_mnli_fever_anli_R1_R2_R3-nli"

tokenizer = AutoTokenizer.from_pretrained(hg_model_hub_name)
model = AutoModelForSequenceClassification.from_pretrained(hg_model_hub_name)

Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def get_prob_for_this_pair(premise, hypothesis, max_length):
  tokenized_input_seq_pair = tokenizer.encode_plus(premise, hypothesis,
                                                     max_length=max_length,
                                                     return_token_type_ids=True, truncation=True)
  input_ids = torch.Tensor(tokenized_input_seq_pair['input_ids']).long().unsqueeze(0)
  # remember bart doesn't have 'token_type_ids', remove the line below if you are using bart.
  token_type_ids = torch.Tensor(tokenized_input_seq_pair['token_type_ids']).long().unsqueeze(0)
  attention_mask = torch.Tensor(tokenized_input_seq_pair['attention_mask']).long().unsqueeze(0)
  outputs = model(input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    labels=None)
    # Note:
    # "id2label": {
    #     "0": "entailment",
    #     "1": "neutral",
    #     "2": "contradiction"
    # },

  predicted_probability = torch.softmax(outputs[0], dim=1)[0].detach().numpy()  # batch_size only one
  print("Premise:", premise)
  print("Hypothesis:", hypothesis)
  print("Entailment:", predicted_probability[0])
  print("Neutral:", predicted_probability[1])
  print("Contradiction:", predicted_probability[2])

  return predicted_probability

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

max_length = 256
labels = []
label_mapping = ['contradiction', 'entailment', 'neutral']

for d in df.to_dict(orient='records'):
  forward_score = get_prob_for_this_pair(d['Premise'], d['Hypothesis'], max_length)
  rev_score = get_prob_for_this_pair(d['Hypothesis'], d['Premise'], max_length)
  this_max = forward_score.argmax()
  print(this_max)
  if this_max == 0:
    labels.append("FE")
  elif this_max == 2:
    labels.append("Negation")
  else:
    if rev_score.argmax() == 0:
      labels.append("RE")
    else:
      labels.append("Neutral")

Streaming output truncated to the last 5000 lines.
Contradiction: 0.0068505285
Premise: A boy is standing in all but one water
Hypothesis: A boy is standing in all but one cold water
Entailment: 0.0078022485
Neutral: 0.9907907
Contradiction: 0.001407098
0
Premise: A boy is standing in every cold water
Hypothesis: A boy is standing in the water

Entailment: 0.85668385
Neutral: 0.13521908
Contradiction: 0.008097046
Premise: A boy is standing in the water

Hypothesis: A boy is standing in every cold water
Entailment: 0.025727207
Neutral: 0.9555523
Contradiction: 0.018720588
0
Premise: A boy is standing in the cold water
Hypothesis: A boy is standing in every water
Entailment: 0.35655412
Neutral: 0.50692403
Contradiction: 0.13652182
Premise: A boy is standing in every water
Hypothesis: A boy is standing in the cold water
Entailment: 0.029868288
Neutral: 0.9663783
Contradiction: 0.003753502
1
Premise: A boy is standing in every cold water
Hypothesis: A boy is standing in every water
Entailm

In [ ]:
df['ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli'] = labels

In [ ]:
len(labels)

1345

In [ ]:
df.head()

,Unnamed: 0,SICK_id,Premise,Hypothesis,Modifier,Premise/Hypothesis/Both,Part of Premise/Hypothesis Modified,Ground Truth,GT,Quantifier Type,nli-deberta-v3-base,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli
0,1,90,a man is jumping into an empty pool,a man is jumping into a full pool,NONE,NONE,NONE,Alternation,Alternation,NONE,Negation,Negation
1,2,90,every man is jumping into an empty pool,a man is jumping into a full pool,every,Premise,Subject,Alternation,Alternation,Universal,Negation,Negation
2,3,90,a man is jumping into an empty pool,every man is jumping into a full pool,every,Hypothesis,Subject,Negation,Negation,Universal,Negation,Negation
3,4,90,every man is jumping into an empty pool,every man is jumping into a full pool,every,Both,Subject,Negation,Negation,Universal,Negation,Negation
4,5,90,some man is jumping into an empty pool,a man is jumping into a full pool,some,Premise,Subject,Alternation,Alternation,Existential,Negation,Negation


## sentence-transformers/nli-roberta-large

In [ ]:
res = CrossEncoder('sentence-transformers/nli-roberta-large').predict(mod_tuple_list)
len(res)

Downloading:   0%|          | 0.00/673 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Some weights of the model checkpoint at sentence-transformers/nli-roberta-large were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/nli-roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this mo

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

1345

In [ ]:
res

## sentence-transformers/nli-mpnet-base-v2

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/nli-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/nli-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)


In [ ]:
sentence_embeddings.shape

## pair-classification-decomposable-attention-elmo




In [ ]:
from allennlp_models.pretrained import get_pretrained_models, get_tasks, load_predictor


predictor = load_predictor("pair-classification-decomposable-attention-elmo")
print(predictor.predict_json({"premise": "An interplanetary spacecraft is in orbit around a gas giant's icy moon.","hypothesis": "The spacecraft has the ability to travel between planets.",}))

In [ ]:
import numpy as np
labels = []
label_mapping = ['entailment','contradiction', 'neutral']
for d in df.to_dict(orient='records'):
  res = np.array(predictor.predict_json({"premise": d['Premise'],"hypothesis": d['Hypothesis']})).argmax()
  rev_res = np.array(predictor.predict_json({"premise": d['Hypothesis'] ,"hypothesis": d['Premise']})).argmax()
  if res == 0:
    labels.append("FE")
  elif res == 1:
    labels.append("Negation")
  else:
    if rev_res == 0:
      labels.append("RE")
    else:
      labels.append("Neutral")

In [ ]:
len(labels)

1345

In [ ]:
df['pair-classification-decomposable-attention-elmo'] = labels

In [ ]:
df.head()

,Unnamed: 0,SICK_id,Premise,Hypothesis,Modifier,Premise/Hypothesis/Both,Part of Premise/Hypothesis Modified,Ground Truth,GT,Quantifier Type,nli-deberta-v3-base,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pair-classification-decomposable-attention-elmo
0,1,90,a man is jumping into an empty pool,a man is jumping into a full pool,NONE,NONE,NONE,Alternation,Alternation,NONE,Negation,Negation,FE
1,2,90,every man is jumping into an empty pool,a man is jumping into a full pool,every,Premise,Subject,Alternation,Alternation,Universal,Negation,Negation,FE
2,3,90,a man is jumping into an empty pool,every man is jumping into a full pool,every,Hypothesis,Subject,Negation,Negation,Universal,Negation,Negation,FE
3,4,90,every man is jumping into an empty pool,every man is jumping into a full pool,every,Both,Subject,Negation,Negation,Universal,Negation,Negation,FE
4,5,90,some man is jumping into an empty pool,a man is jumping into a full pool,some,Premise,Subject,Alternation,Alternation,Existential,Negation,Negation,FE


In [ ]:
# set_129 = df.rename(columns={"label":"Ground Truth"})
# set_129.head()

In [ ]:
df.to_excel(os.path.join(path,"zero-shot-apr21_annotations_nli_eval.xlsx"))

In [ ]:
df['CompressedGT'] = df['Ground Truth']
df['CompressedGT'][ df['CompressedGT'] == 'Alternation' ] = "Contradiction"
df['CompressedGT'][ df['CompressedGT'] == 'Negation' ] = "Contradiction"
df['CompressedGT'][ df['CompressedGT'] == 'Negation|Alternation' ] = "Contradiction"
d = df[(df['CompressedGT'] == 'Contradiction') | (df['CompressedGT'] == 'Neutral') | (df['CompressedGT'] == 'FE') | (df['CompressedGT'] == 'RE')]
d['nli-deberta-v3-base'][d['nli-deberta-v3-base'] == "Negation"] = "Contradiction"
d['ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli'][d['ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli'] == "Negation"] = "Contradiction"
d['pair-classification-decomposable-attention-elmo'][d['pair-classification-decomposable-attention-elmo'] == "Negation"] = "Contradiction"

<ipython-input-39-085a7c5a8065>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CompressedGT'][ df['CompressedGT'] == 'Alternation' ] = "Contradiction"
<ipython-input-39-085a7c5a8065>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CompressedGT'][ df['CompressedGT'] == 'Negation' ] = "Contradiction"
<ipython-input-39-085a7c5a8065>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CompressedGT'][ df['CompressedGT'] == 'Nega

In [ ]:
df['CompressedGT'].unique()

array(['Contradiction', 'Neutral', 'FE', 'RE', 'Cover|FE', 'Equivalence',
       'Cover'], dtype=object)

In [ ]:
len(df[ (df['CompressedGT'] != 'Cover|FE') & (df['CompressedGT'] != 'Equivalence')])

1337

In [ ]:
df[ (df['CompressedGT'] != 'Cover|FE') & (df['CompressedGT'] != 'Equivalence')]

In [ ]:
len(df[df['CompressedGT'] == 'Cover|FE']), len(df[df['CompressedGT'] == 'Equivalence']), 

(1, 7)

In [ ]:
# from pycm import *
# cm = ConfusionMatrix(actual_vector=df['CompressedGT'].to_list(), predict_vector=df['nli-deberta-v3-base'].to_list())
# cm.table

In [ ]:
# cm.print_matrix()

In [ ]:
cm.stat(summary=True)

Overall Statistics : 

ACC Macro                                                         0.88525
F1 Macro                                                          0.42323
FPR Macro                                                         0.10637
Kappa                                                             0.536
Overall ACC                                                       0.77049
PPV Macro                                                         None
SOA1(Landis & Koch)                                               Moderate
TPR Macro                                                         0.61834
Zero-one Loss                                                     28

Class Statistics :

Classes                                                           FE            Negation      Neutral       RE            
ACC(Accuracy)                                                     0.86885       0.91803       0.80328       0.95082       
AUC(Area under the ROC curve)                        